# This code reads the DL3 files of LST-1 data, gets the IRFs and background from them and saves them in the needed format for the simualtion

In [1]:
import numpy as np
from astropy.coordinates import SkyCoord
from regions import PointSkyRegion
from gammapy.data import DataStore
from gammapy.maps import MapAxis, RegionGeom, Map
from gammapy.datasets import (
    Datasets,
    SpectrumDataset,

)
from gammapy.makers import (
    SpectrumDatasetMaker,
    WobbleRegionsFinder,
    ReflectedRegionsBackgroundMaker,
    SafeMaskMaker,
)

In [2]:
# load the DL3 files
dir_path = "testData/" #change to the path where you have the data

data_store = DataStore.from_dir(dir_path)
observations = data_store.get_observations(
    required_irf=["aeff", "edisp", "rad_max"]
)

Run the below cell only if you don't have the index files (you need to have lstchain installed)

In [3]:
# !lstchain_create_dl3_index_files -d $dir_path --overwrite  #do first time you use the data.

In [4]:
obj_name = np.unique(data_store.obs_table["OBJECT"])[0]
target_position = SkyCoord.from_name(obj_name, frame='icrs')
on_region = PointSkyRegion(target_position)

Set the migration matrix axis

In [5]:
# true and estimated energy axes
energy_axis = MapAxis.from_energy_bounds(
    50, 1e5, nbin=5, per_decade=True, unit="GeV", name="energy"
)
energy_axis_true = MapAxis.from_energy_bounds(
    10, 1e5, nbin=10, per_decade=True, unit="GeV", name="energy_true"
)

# geometry defining the ON region and SpectrumDataset based on it
geom = RegionGeom.create(region=on_region, axes=[energy_axis])

dataset_empty = SpectrumDataset.create(
    geom=geom, energy_axis_true=energy_axis_true
)


In [6]:
dataset_maker = SpectrumDatasetMaker(
    containment_correction=False, selection=["counts", "exposure", "edisp"]
)

region_finder = WobbleRegionsFinder(n_off_regions=1) #select the number of OFF regions, in this case we use 1 (use the same as the one in the DL3 files)
bkg_maker = ReflectedRegionsBackgroundMaker(region_finder=region_finder)

# use the energy threshold specified in the DL3 files
safe_mask_masker = SafeMaskMaker(methods=["aeff-default"])


In [7]:
datasets = Datasets()

# create a counts map for visualisation later...
counts = Map.create(skydir=target_position, width=3)

for observation in observations:
    dataset = dataset_maker.run(
        dataset_empty.copy(name=str(observation.obs_id)), observation
    )
    counts.fill_events(observation.events)
    dataset_on_off = bkg_maker.run(dataset, observation)
    dataset_on_off = safe_mask_masker.run(dataset_on_off, observation)
    datasets.append(dataset_on_off)


No default upper safe energy threshold defined for obs 5447
No default lower safe energy threshold defined for obs 5447


## Outputs we want

In [8]:
background_for_txt = datasets[0].background.data[:,0][:,0] #change [0] in all lines of this cell for the dl3 file you want to produce the data from.
time = data_store.obs_table["LIVETIME"][0]
migmat_for_txt = (datasets[0].edisp.get_edisp_kernel().data.T * datasets[0].exposure.data[:,0][:,0]).T
Etrue = energy_axis_true.as_plot_center
Eest = energy_axis.as_plot_center
migmatxEtrue = energy_axis_true.edges
migmatyEest = energy_axis.edges
print("observation time: " + str(time) + ' s')
excess = datasets[0].excess.data[:,0][:,0]

observation time: 1144.492552611293 s


In [9]:
#save it in the path that you want
np.savetxt('data_lst-1/migmatval.txt', migmat_for_txt)
np.savetxt('data_lst-1/bckg.txt', background_for_txt)
np.savetxt('data_lst-1/migmatxEtrue.txt', migmatxEtrue.to_value())
np.savetxt('data_lst-1/migmatyEest.txt', migmatyEest.to_value())
np.savetxt('data_lst-1/Eest.txt', Eest.to_value())
np.savetxt('data_lst-1/Etrue.txt', Etrue.to_value())